In [1]:
from config import gkey
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import warnings
gmaps.configure(api_key=gkey)
warnings.filterwarnings('ignore')


In [2]:
path="C:/Users/rwang/Desktop/python-api-challenge/WeatherPy/retrieved data.csv"
file=pd.read_csv(path)
data=pd.DataFrame(file)
data.sort_values(by=["temp"])

,Unnamed: 0,lat,lon,city,temp,humidity,windspeed,cloudness
69,99,72.018,-86.025,clyde river,249.15,83.0,1.50,90.0
82,122,81.167,-105.905,yellowknife,251.31,63.0,2.10,5.0
4,4,87.217,-93.178,qaanaaq,252.70,98.0,1.90,100.0
106,187,74.924,-58.650,upernavik,255.20,71.0,4.48,100.0
44,58,79.521,-145.315,barrow,258.15,77.0,1.50,1.0
...,...,...,...,...,...,...,...,...
217,469,-4.856,25.770,lubao,306.04,52.0,1.00,75.0
476,1344,14.557,79.946,kovur,306.50,41.0,5.36,0.0
195,412,25.150,76.456,baran,307.00,18.0,2.45,32.0
5,6,25.780,65.582,bela,307.00,18.0,0.57,18.0


In [3]:
locations=data[["lat","lon"]]
humidity=data["humidity"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,weights=humidity,max_intensity=10,point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
newdata=data[(data["temp"]>200)&(data["temp"]<300)&(data["windspeed"]<5)&(data["humidity"]<50)&(data["cloudness"]==0)]
newdata.reset_index(inplace=True)
newdata

,index,Unnamed: 0,lat,lon,city,temp,humidity,windspeed,cloudness
0,37,49,25.279,61.098,chabahar,295.20,45.0,4.30,0.0
1,50,72,56.369,31.352,toropets,271.10,43.0,1.39,0.0
2,71,102,28.026,56.174,qeshm,291.67,48.0,2.10,0.0
3,112,200,16.047,55.712,salalah,298.15,44.0,3.10,0.0
4,134,264,51.253,28.564,luhyny,271.40,45.0,4.81,0.0
5,140,275,23.858,48.394,riyadh,286.15,43.0,1.00,0.0
6,168,341,7.742,16.907,beboto,299.90,43.0,1.74,0.0
7,271,634,26.728,-102.663,ocampo,290.00,34.0,1.61,0.0
8,275,650,26.136,-110.415,loreto,293.70,29.0,2.47,0.0
9,334,820,57.434,37.151,kesova gora,271.00,39.0,4.33,0.0


In [5]:
newdata["Hotel Name"] = ""
newdata["Hotel city"] = ""
newdata["Hotel Coutry"] = ""
newdata

,index,Unnamed: 0,lat,lon,city,temp,humidity,windspeed,cloudness,Hotel Name,Hotel city,Hotel Coutry
0,37,49,25.279,61.098,chabahar,295.20,45.0,4.30,0.0,,,
1,50,72,56.369,31.352,toropets,271.10,43.0,1.39,0.0,,,
2,71,102,28.026,56.174,qeshm,291.67,48.0,2.10,0.0,,,
3,112,200,16.047,55.712,salalah,298.15,44.0,3.10,0.0,,,
4,134,264,51.253,28.564,luhyny,271.40,45.0,4.81,0.0,,,
5,140,275,23.858,48.394,riyadh,286.15,43.0,1.00,0.0,,,
6,168,341,7.742,16.907,beboto,299.90,43.0,1.74,0.0,,,
7,271,634,26.728,-102.663,ocampo,290.00,34.0,1.61,0.0,,,
8,275,650,26.136,-110.415,loreto,293.70,29.0,2.47,0.0,,,
9,334,820,57.434,37.151,kesova gora,271.00,39.0,4.33,0.0,,,


In [6]:
params = {
    "radius": 5000,
    "types": "hotel",
    "key": gkey
}
lat=newdata["lat"]
lon=newdata["lon"]
params["location"] = f"{lat},{lon}"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
name_address = requests.get(base_url, params=params)
name_address = name_address.json()
try:
        newdata.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        newdata.loc[index, "Hotel City"] = name_address["results"][0]["address_components"][4]["short_name"]
        newdata.loc[index, "Hotel Coutry"] = name_address["results"][0]["address_components"][6]["long_name"]
except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [ ]:
locations1=newdata[["lat","lon"]]
humidity1=newdata["humidity"].astype(float)
fig1 = gmaps.figure()
heat_layer1 = gmaps.heatmap_layer(locations1,weights=humidity1,max_intensity=10,point_radius=2)
fig1.add_layer(heat_layer1)

hotelname=newdata["Hotel Name"]
hotelcountry=newdata["Hotel Coutry"]
markers= gmaps.marker_layer(hotelname)
markers = gmaps.marker_layer(hotelcountry)
fig1.add_layer(markers)
fig1